# text to video synthesis

O repositório original está: https://modelscope.cn/models/damo/text-to-video-synthesis/summary

Repositório do Hugging Face: https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis

In [1]:
!pip install huggingface_hub open_clip_torch modelscope pytorch_lightning transformers -q

In [ ]:
import os
import pathlib
import random

import torch
from torchvision.io import write_video
from huggingface_hub import snapshot_download

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


if os.getenv('SYSTEM') == 'spaces':
    os.system('pip uninstall -y modelscope')
    os.system(
        'pip install git+https://github.com/modelscope/modelscope.git@refs/pull/207/head'
    )

from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline

model_dir = pathlib.Path('weights')
if not model_dir.exists():
    model_dir.mkdir()
    snapshot_download('damo-vilab/modelscope-damo-text-to-video-synthesis',
                      repo_type='model',
                      local_dir=model_dir)

pipe = pipeline('text-to-video-synthesis', model_dir.as_posix())

2023-03-20 01:36:45,077 - modelscope - INFO - PyTorch version 1.13.1+cu116 Found.
2023-03-20 01:36:45,091 - modelscope - INFO - TensorFlow version 2.11.0 Found.
2023-03-20 01:36:45,102 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-03-20 01:36:45,186 - modelscope - INFO - Loading done! Current index file version is 1.4.1, with md5 c99af2abbaffaa749d9fa0ce360b29fd and a total number of 842 components indexed
2023-03-20 01:36:55,356 - modelscope - INFO - initiate model from weights
2023-03-20 01:36:55,358 - modelscope - INFO - initiate model from location weights.
2023-03-20 01:36:55,363 - modelscope - INFO - initialize model from weights


In [ ]:
def generate(prompt: str, seed: int) -> torch.Tensor:
    if seed == -1:
        seed = random.randint(0, 1000000)
    torch.manual_seed(seed)
    output = pipe({'text': prompt})[OutputKeys.OUTPUT_VIDEO]
    output_tensor = torch.from_numpy(output).to(device).permute(3, 0, 1, 2)
    return output_tensor

# Testando a função generate
output = generate("An astronaut riding a horse.", 0)
print(output.size())

# Definindo exemplo
examples = [
    ['An astronaut riding a horse.', 0],
    ['A panda eating bamboo on a rock.', 0],
    ['Spiderman is surfing.', 0],
]

In [ ]:
# Função que salva o tensor de vídeo como um arquivo mp4
def save_video(tensor: torch.Tensor, filename: str):
    video = tensor.permute(1, 2, 3, 0).cpu().numpy()
    write_video(filename, video, fps=25)


# Testando a função save_video
save_video(output, "output.mp4")